# Análisis del riesgo de incumplimiento de los prestatarios

El proyecto consiste en preparar un informe para la división de préstamos de un banco. Averiguaremos si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

El informe tendrá en cuenta la **puntuación de crédito** para un cliente potencial. 
(La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo)



# Descripción de los datos

- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo



# Importamos librerias y cargamos los datos



In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/credit_scoring_eng.csv')

# Exploración de los datos

In [3]:
#vamos a mostrar las primeras filas 
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [4]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
df.shape

(21525, 12)

In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Vemos en la exploracion inicial de los datos las siguientes inconsistencias:**
- En la columna de `days_employed`, no tienen sentido los valores negativos y el entero máximo (son muchísimos los días para que una persona pueda trabajar en vida)
- En la columna `children` el valor -1 tampoco tiene sentido 
- Tenemos valores ausentes en dos columnas; `days_employed` y `total_income`.

De acuerdo a la información tengo 21525 registros.

En `days_employed` y en `total_income` hay la misma cantidad de valores ausentes `2174`

In [8]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
df_na = df[df['days_employed'].isna()]

In [9]:
df_na

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Parece que los valores son simétricos en la columna `'days_employed'` y `'total_income'`.

Tambien parece haber un patrón con la columna de educación y valores ausentes! 
Vamos a confirmar esta teoría!

In [10]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
df[(df['days_employed'].isna()) & (df['total_income'].isna() )]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Hay 2174 filas de valores ausentes en ambas, eso quiere decir que si son simetricos!

In [11]:
df[(df['days_employed'].isna()) & (df['education_id'] == 1)]




,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21426,0,NaN,49,secondary education,1,married,0,F,employee,1,NaN,property
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [12]:
# Comprobando la distribución en el conjunto de datos entero
df['education_id'].value_counts(normalize=True)

education_id
1    0.707689
0    0.244367
2    0.034564
3    0.013101
4    0.000279
Name: proportion, dtype: float64

In [13]:
# Comprobación de la distribución
df_na['education_id'].value_counts(normalize=True)

education_id
1    0.708372
0    0.250230
2    0.031739
3    0.009660
Name: proportion, dtype: float64

In [14]:
df['family_status_id'].value_counts(normalize=True)

family_status_id
0    0.575145
1    0.194053
4    0.130685
3    0.055517
2    0.044599
Name: proportion, dtype: float64

In [15]:
df_na['family_status_id'].value_counts(normalize=True)

family_status_id
0    0.568997
1    0.203312
4    0.132475
3    0.051518
2    0.043698
Name: proportion, dtype: float64

In [16]:
df['income_type'].value_counts(normalize=True)

income_type
employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: proportion, dtype: float64

In [17]:
df_na['income_type'].value_counts(normalize=True)

income_type
employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: proportion, dtype: float64

In [18]:
df['gender'].value_counts(normalize=True)

gender
F      0.661370
M      0.338583
XNA    0.000046
Name: proportion, dtype: float64

In [19]:
df_na['gender'].value_counts(normalize=True)

gender
F    0.682613
M    0.317387
Name: proportion, dtype: float64

**Conclusión inicial**

Parece no haber patrones en los valores ausentes, pudo solo ser una mala captura a la hora del registro de los datos ya que de acuerdo a la distribucion el porcentaje no cambia mucho.

**El filtrado sin valores ausentes y la tabla con valores ausentes; son prácticamente el mismo porcentaje de valores**

In [20]:
# Vamos a investigar a los clientes que no tienen datos sobre las característica identificada y la columna con los valores ausentes
valores_ausentes = 2174
data_completa = 21525
porcentaje_valores_ausentes = valores_ausentes/data_completa * 100
print(f'porcentaje de valores ausentes: {porcentaje_valores_ausentes}')


porcentaje de valores ausentes: 10.099883855981417


# Transformación de datos



**education**

In [21]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir tipograficamente
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [22]:
# Correccion de registro
correcciones = {
    'SECONDARY EDUCATION': 'secondary education',
    "BACHELOR'S DEGREE" : 'bachelor\'s degree',
    'Secondary Education': 'secondary education',
    "Bachelor's Degree": 'bachelor\'s degree',
    'SOME COLLEGE': 'some college',
    'Some College': 'some college',
    'PRIMARY EDUCATION': 'primary education',
    'Primary Education': 'primary education',
     'Graduate Degree': 'graduate degree',
    'GRADUATE DEGREE': 'graduate degree'           
}
df['education'].replace(correcciones, inplace=True)


In [23]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
df['education'].unique()



array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

**children**

In [24]:
# Veamos la distribución de los valores en la columna `children`
df['children'].unique()


array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [25]:
df['children'].value_counts()

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64

En la columna `'children'` hay un valor -1 probablemente el niño falleciÓ y antes pudieron tener un hijo, pero actualmente no tienen.Probablemente a eso se pudieran haber referido, en todo caso -1 querria decir que no tienen hijos, por lo tanto el valor es 0.

Tambien el valor 20 probablemente fue un error de captura en los datos! hoy en dia esa cantidad de hijos es atípica, lo más lógico sería pensar que son 2 en lugar de '20'. 
Cambiaremos esos datos a sus valores correctos!.

In [26]:
# coorigiendo los valores
diccionario ={
    -1: 0,
    20: 2
}
df['children'].replace(diccionario, inplace=True)


In [27]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
df['children'].unique()



array([1, 0, 3, 2, 4, 5], dtype=int64)

**days_employed**

In [28]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [29]:
# Encontramos datos problemáticos en `days_employed` y calculamos el porcentaje
df['days_employed'].isna().sum() / df.shape[0]






0.10099883855981417

In [30]:
valores_negativos = df['days_employed'] <0 
df.loc[valores_negativos, 'days_employed'] = df['days_employed'].abs()


In [31]:
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [32]:
df.loc[df['days_employed']>0, 'days_employed'] =  df.loc[df['days_employed']>0, 'days_employed'] / 100
df['days_employed'] = df['days_employed'].abs() 

In [33]:
df['days_employed'].describe()

count    19351.000000
mean       669.147289
std       1390.308805
min          0.241416
25%          9.270093
50%         21.942206
75%         55.378824
max       4017.554005
Name: days_employed, dtype: float64

La cantidad de datos es alta del 10% podría deberse a problemas técnicos
- Datos nulos: podemos solucionarlo más adelante.
- Datos negativos: los transformamos en positivos.
- Datos positivos muy grandes: solucionamos el problema pensando que esta en escala(que estuvieran multiplicados por 100)

**dob_years**

In [34]:
# Revisando `dob_years` en busca de valores sospechosos y contando el porcentaje
df['dob_years'].value_counts(normalize=True)



dob_years
35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
42    0.027735
33    0.026992
39    0.026620
31    0.026016
36    0.025784
44    0.025412
29    0.025319
30    0.025087
48    0.024994
37    0.024948
50    0.023879
43    0.023833
32    0.023693
49    0.023600
28    0.023368
45    0.023089
27    0.022904
56    0.022625
52    0.022485
47    0.022300
54    0.022253
46    0.022067
58    0.021417
57    0.021370
53    0.021324
51    0.020813
59    0.020627
55    0.020581
26    0.018955
60    0.017515
25    0.016585
61    0.016492
62    0.016353
63    0.012497
64    0.012311
24    0.012265
23    0.011800
65    0.009013
22    0.008502
66    0.008502
67    0.007758
21    0.005157
0     0.004692
68    0.004599
69    0.003949
70    0.003020
71    0.002695
20    0.002369
72    0.001533
19    0.000650
73    0.000372
74    0.000279
75    0.000046
Name: proportion, dtype: float64

- Podemos observar que hay valores de 0 en la edad de los clientes y representa el 04% del total de los valores de la columna `dob_years`. 
- Observamos que el mayor porcentaje corresponde a la edad de 35 años
- Observamos que el menor porcentaje es de 75 años

In [35]:
#Vamos a ver el conjunto de datos entero de una manera más general con el metodo .describe()
df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Podemos ver que la media es de 43 y la mediana de 42 años asi que corregiremos la edad de 0 años con la media

Vamos a cambiar el valor 0 años con la media de edad de la tabla

In [36]:
df['dob_years'].replace({0: 43}, inplace=True)


In [37]:
# Comprobando que esta arreglado
df['dob_years'].describe()


count    21525.000000
mean        43.495145
std         12.218213
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

**family_status**

In [38]:
# Veamos los valores de la columna
df['family_status'].unique()



array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [39]:
# Abordando los valores problemáticos en `family_status`
df['family_status'].replace({'civil partnership': 'free union'}, inplace=True)


Cambiamos el valor de 'civil partnership' por 'free union'(me imagino es la traducción en español union libre)

In [40]:
# Comprobamos que este arreglado
df['family_status'].unique()


array(['married', 'free union', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

**gender**

In [41]:
# Veamos los valores en la columna
df['gender'].value_counts()

gender
F      14236
M       7288
XNA        1
Name: count, dtype: int64

el valor XNA quien sabe que sea pero solo es 'uno' de acuerdo al conteo de los valores asi que lo cambiaremos a femenino ya que es el que de acuerdo al conteo es el mayor número en la columna.

In [42]:
# Abordamos los valores problemáticos
df['gender'].replace({'XNA': 'F'}, inplace=True)



In [43]:
# Comprobamos el resultado - aseguramos de que esté arreglado
df['gender'].value_counts()


gender
F    14237
M     7288
Name: count, dtype: int64

**income_type**

In [44]:
# Veamos los valores en la columna
df['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

Vamos a juntar al funcionario en la misma categoría que al empleado ya que es un empleo, al que tiene negocio lo juntaremos con emprendedor ya que un negocio es emprender y dejaremos los demás como estan.

In [45]:
# Abordamos los valores problemáticos, si existen
df['income_type'].replace({
    'civil servant': 'employee',
    'business': 'entrepreneur'
},inplace=True)

In [46]:
# Comprobamos el resultado si esta arreglado
df['income_type'].unique()


array(['employee', 'retiree', 'entrepreneur', 'unemployed', 'student',
       'paternity / maternity leave'], dtype=object)

## Ahora veamos si hay duplicados en nuestros datos. 

In [47]:
#Comprobar los duplicados
df.duplicated().sum()


74

hay 74 valores duplicados en toda nuestra tabla.Vamos a eliminarlos 

In [48]:
# Abordamos los duplicados y resetearemos el indice para que este en orden el indice
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [49]:
#comprobación para ver si tenemos duplicados
df.duplicated().sum()

0

In [50]:
df.shape

(21451, 12)

## Asignaremos valores más descriptivos a las columnas  'education_id' y 'family_status_id'

In [51]:
df['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [52]:
df['family_status_id'].unique()


array([0, 1, 2, 3, 4], dtype=int64)

In [53]:
diccionario = {
    'education_id': {
        0: 'primary education',
        1: 'secondary education',
        2: 'some college',
        3: 'bachelor\'s degree',
        4: 'graduate degree'
    },
    'family_status_id': {
        0: 'unmarried',
        1: 'married',
        2: 'divorced',
        3: 'widow / widower',
        4: 'free union'
    }
}

**Conclusión intermedia**
- Corregimos registros cambiando valores que no tenian sentido 
- Eliminamos duplicados
- clasificamos mejor ciertos valores a un mismo valor sinonimo. 

Ahora contamos con datos casi limpios de 21451 filas y 12 columnas! Falta trabajar con los valores ausentes

# Trabajar con valores ausentes

## Restaurar valores ausentes en `total_income`

`total_income` tiene valores ausentes.Abordaremos estos valores. 
- Crearemos una `categoría de edad` para los clientes asi nos podrá a ayudar a calcular valores para el ingreso total.

In [54]:
# Vamos a escribir una función que calcule la categoría de edad
def edad_category (edad):
    if edad <18:
        return 'menor de edad'
    elif edad >= 18 and edad < 35:
        return 'adulto joven'
    elif edad >= 35 and edad < 45:
        return 'adulto'
    elif edad >= 45 and edad <65:
        return 'adulto mayor'
    else:
        return 'tercera edad'
        
    
    

In [55]:
# Probando si la función funciona bien
edad_category(90)


'tercera edad'

In [56]:
# Nueva columna basada en la función
df['edad_category'] = df['dob_years'].apply(edad_category)


In [57]:
#Comprobando los nuevos valores
df[['dob_years', 'edad_category']]


,dob_years,edad_category
0,42,adulto
1,36,adulto
2,33,adulto joven
3,32,adulto joven
4,53,adulto mayor
...,...,...
21446,43,adulto
21447,67,tercera edad
21448,38,adulto
21449,38,adulto


In [58]:
# Creando una tabla sin valores ausentes y mostrando algunas de sus filas para asegurarnos de que se ve bien
df_new = df.dropna()
df_new
    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,edad_category
0,1,84.376730,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adulto
1,1,40.248038,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adulto
2,0,56.234226,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adulto joven
3,3,41.247472,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adulto joven
4,0,3402.660720,53,secondary education,1,free union,1,F,retiree,0,25378.572,to have a wedding,adulto mayor
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,1,45.293167,43,secondary education,1,free union,1,F,entrepreneur,0,35966.698,housing transactions,adulto
21447,0,3439.374041,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,tercera edad
21448,1,21.133469,38,secondary education,1,free union,1,M,employee,1,14347.610,property,adulto
21449,3,31.124817,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adulto


In [59]:
df_new.head(13)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,edad_category
0,1,84.376730,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adulto
1,1,40.248038,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adulto
2,0,56.234226,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adulto joven
3,3,41.247472,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adulto joven
4,0,3402.660720,53,secondary education,1,free union,1,F,retiree,0,25378.572,to have a wedding,adulto mayor
5,0,9.261858,27,bachelor's degree,0,free union,1,M,entrepreneur,0,40922.170,purchase of the house,adulto joven
6,0,28.792021,43,bachelor's degree,0,married,0,F,entrepreneur,0,38484.156,housing transactions,adulto
7,0,1.527796,50,secondary education,1,married,0,M,employee,0,21731.829,education,adulto mayor
8,2,69.298653,35,bachelor's degree,0,free union,1,F,employee,0,15337.093,having a wedding,adulto
9,0,21.887564,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adulto


In [60]:
df.loc[12:12]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,edad_category
12,0,NaN,65,secondary education,1,free union,1,M,retiree,0,NaN,to have a wedding,tercera edad


In [61]:
df_new.shape

(19351, 13)

en la nueva tabla sin valores ausentes contamos con '19351' filas y '13'columnas.

In [62]:
median_by_group = df_new.groupby('education')['total_income'].median().reset_index().rename(columns={'total_income': 'total_income_median'})

In [63]:
#una tabla agrupadas por la educacion con su mediana del total de ingresos!!
median_by_group

,education,total_income_median
0,bachelor's degree,28054.5310
1,graduate degree,25161.5835
2,primary education,18741.9760
3,secondary education,21836.5830
4,some college,25618.4640


In [64]:
df = df.merge(median_by_group, on='education', how='left')


In [65]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,edad_category,total_income_median
0,1,84.376730,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adulto,28054.531
1,1,40.248038,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adulto,21836.583
2,0,56.234226,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adulto joven,21836.583
3,3,41.247472,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adulto joven,21836.583
4,0,3402.660720,53,secondary education,1,free union,1,F,retiree,0,25378.572,to have a wedding,adulto mayor,21836.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,1,45.293167,43,secondary education,1,free union,1,F,entrepreneur,0,35966.698,housing transactions,adulto,21836.583
21447,0,3439.374041,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,tercera edad,21836.583
21448,1,21.133469,38,secondary education,1,free union,1,M,employee,1,14347.610,property,adulto,21836.583
21449,3,31.124817,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adulto,21836.583


In [66]:
def completar_ausentes1(df, nombre_columna):
    if df[nombre_columna].isna().any():
        df[nombre_columna].fillna(df['total_income_median'], inplace=True)
    return df

In [67]:
completar_ausentes1(df,'total_income')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,edad_category,total_income_median
0,1,84.376730,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adulto,28054.531
1,1,40.248038,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adulto,21836.583
2,0,56.234226,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adulto joven,21836.583
3,3,41.247472,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adulto joven,21836.583
4,0,3402.660720,53,secondary education,1,free union,1,F,retiree,0,25378.572,to have a wedding,adulto mayor,21836.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,1,45.293167,43,secondary education,1,free union,1,F,entrepreneur,0,35966.698,housing transactions,adulto,21836.583
21447,0,3439.374041,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,tercera edad,21836.583
21448,1,21.133469,38,secondary education,1,free union,1,M,employee,1,14347.610,property,adulto,21836.583
21449,3,31.124817,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adulto,21836.583


In [68]:
df.loc[12:12]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,edad_category,total_income_median
12,0,NaN,65,secondary education,1,free union,1,M,retiree,0,21836.583,to have a wedding,tercera edad,21836.583


Acabamos de completar los valores ausentes  en `'total_income'`, ahora podemos ver las '21451' filas autocompletadas con la **mediana**

In [71]:
# Comprueba si tenemos algún error
df['total_income'].isna().sum()

0

listo ya no hay valores nulos en `total_income`.Utilizamos dos métodos juntos para comprobarlo.

##  Restaurar valores en `days_employed`

In [72]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados

median_by_edad = df_new.groupby('edad_category')['days_employed'].median().reset_index().rename(columns={'days_employed':'days_employed_median'})

In [73]:
#Creamos una tabla para observar la mediana de los dias laborados por la 'categoria de edad'
median_by_edad

,edad_category,days_employed_median
0,adulto,18.560189
1,adulto joven,11.785109
2,adulto mayor,43.346828
3,tercera edad,3603.042323


In [74]:
df = df.merge(median_by_edad, on='edad_category', how='left')


In [75]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'edad_category', 'total_income_median',
       'days_employed_median'],
      dtype='object')

In [76]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado
def completar_ausentes2(df, nombre_columna):
    if df[nombre_columna].isna().any():
        df[nombre_columna].fillna(df['days_employed_median'], inplace=True)
    return df

In [77]:
completar_ausentes2(df,'days_employed')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,edad_category,total_income_median,days_employed_median
0,1,84.376730,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adulto,28054.531,18.560189
1,1,40.248038,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adulto,21836.583,18.560189
2,0,56.234226,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adulto joven,21836.583,11.785109
3,3,41.247472,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adulto joven,21836.583,11.785109
4,0,3402.660720,53,secondary education,1,free union,1,F,retiree,0,25378.572,to have a wedding,adulto mayor,21836.583,43.346828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,1,45.293167,43,secondary education,1,free union,1,F,entrepreneur,0,35966.698,housing transactions,adulto,21836.583,18.560189
21447,0,3439.374041,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,tercera edad,21836.583,3603.042323
21448,1,21.133469,38,secondary education,1,free union,1,M,employee,1,14347.610,property,adulto,21836.583,18.560189
21449,3,31.124817,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adulto,21836.583,18.560189


In [78]:
df['days_employed'].isna().sum()



0

Listo ya no hay valores ausentes en `'days_employed'` 

## Clasificación de datos


In [86]:
#Mostrando los valores de los datos seleccionados para la clasificación
df[['purpose']].head(20)



,purpose
0,purchase of the house
1,car purchase
2,purchase of the house
3,supplementary education
4,to have a wedding
5,purchase of the house
6,housing transactions
7,education
8,having a wedding
9,purchase of the house for my family


In [99]:
# Comprobando los valores únicos
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [100]:
#Escribamos una función para clasificar los datos
def clean_purpose (fraseo):
    if ('house' in fraseo) or ('housing' in fraseo) or ('property' in fraseo) or ('real estate' in fraseo):
        return 'Property'
    elif 'car' in fraseo:
        return 'Car'
    elif ('education' in fraseo) or ('university' in fraseo) or ('educated' in fraseo):
        return 'Education'
    elif 'wedding' in fraseo:
        return 'Wedding'
    else: 
        return 'Other'

In [101]:
# Creamos una columna con las categorías y contamos los valores en ellas/distribucion de las nuevas categorías
df['clean_purpose'] = df['purpose'].apply(clean_purpose)
df['clean_purpose'].value_counts(normalize=True)

clean_purpose
Property     0.503893
Car          0.200737
Education    0.187031
Wedding      0.108340
Name: proportion, dtype: float64

In [102]:
df.groupby(['clean_purpose', 'purpose'])['purpose'].count().reset_index(name='count')

,clean_purpose,purpose,count
0,Car,buying a second-hand car,478
1,Car,buying my own car,505
2,Car,car,494
3,Car,car purchase,461
4,Car,cars,478
5,Car,purchase of a car,455
6,Car,second-hand car purchase,486
7,Car,to buy a car,471
8,Car,to own a car,478
9,Education,education,447


In [103]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación
df['children'].value_counts(normalize=True)


children
0    0.658990
1    0.224092
2    0.099203
3    0.015384
4    0.001911
5    0.000420
Name: proportion, dtype: float64

In [104]:
# Obtener estadísticas resumidas para la columna
df['children'].describe()


count    21451.000000
mean         0.478393
std          0.756040
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

In [115]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos
def children_class (cantidad):
    if cantidad == 0 :
        return '0 hijos'
    elif cantidad >= 1 and cantidad <3:
        return '1 ó 2 hijos'
    else:
        return '3 ó más hijos'
    


In [116]:
# Crear una columna con categorías
df['children_class'] = df['children'].apply(children_class)


In [117]:
# Contar los valores de cada categoría para ver la distribución
df['children_class'].value_counts()

children_class
0 hijos          14136
1 ó 2 hijos       6935
3 ó más hijos      380
Name: count, dtype: int64

In [118]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,edad_category,total_income_median,days_employed_median,clean_purpose,children_class
0,1,84.376730,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adulto,28054.531,18.560189,Property,1 ó 2 hijos
1,1,40.248038,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adulto,21836.583,18.560189,Car,1 ó 2 hijos
2,0,56.234226,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adulto joven,21836.583,11.785109,Property,0 hijos
3,3,41.247472,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adulto joven,21836.583,11.785109,Education,3 ó más hijos
4,0,3402.660720,53,secondary education,1,free union,1,F,retiree,0,25378.572,to have a wedding,adulto mayor,21836.583,43.346828,Wedding,0 hijos


**Creamos nuevas caracteristicas**

In [119]:
df['total_income'].describe()

count     21451.000000
mean      26466.223995
std       15702.203660
min        3306.762000
25%       17215.531500
50%       22582.311000
75%       31331.235000
max      362496.645000
Name: total_income, dtype: float64

In [120]:
def total_income_category (cantidad):
    if cantidad >=  3306.762000 and cantidad <  22582.311000:
        return 'bajo'
    elif cantidad >= 22582.311000 and cantidad < 31331.235000:
        return 'medio'
    else: 
        cantidad >=  31331.235000
        return 'alto'
    


In [121]:
df['total_income_category'] = df['total_income'].apply(total_income_category)

In [122]:
df[['total_income', 'total_income_category']]

,total_income,total_income_category
0,40620.102,alto
1,17932.802,bajo
2,23341.752,medio
3,42820.568,alto
4,25378.572,medio
...,...,...
21446,35966.698,alto
21447,24959.969,medio
21448,14347.610,bajo
21449,39054.888,alto


In [123]:
df['total_income_category'].value_counts()

total_income_category
bajo     10725
alto      5363
medio     5363
Name: count, dtype: int64

# Comprobación de las hipótesis


**1ra hipótesis.
¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [124]:
df.pivot_table(index='children_class', values='debt', aggfunc= 'mean')

,debt
children_class,
0 hijos,0.075269
1 ó 2 hijos,0.093151
3 ó más hijos,0.081579


**Conclusión 1ra hipótesis.**

Podemos observar en la tabla, que las personas con hijos del rango de **1 a 2 hijos tienen una tasa de impago más alta** que las personas 0 hijos o incluso más de 3 hijos

**2da hipotesis. ¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [111]:
df.pivot_table(index='family_status', values='debt', aggfunc= 'mean').sort_values(by='debt', ascending=True)

,debt
family_status,
widow / widower,0.065693
divorced,0.071130
married,0.075470
free union,0.093471
unmarried,0.097509


**Conclusión 2da hipótesis.**

Todo lo contrario a lo que pensariamos!! las personas **sin casarse  son las que pagan menos**, y las personas viudas son las que menor deuda tienen!!

**3ra hipótesis ¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [99]:
df.pivot_table(index='total_income_category', values='debt', aggfunc= 'mean')

,debt
total_income_category,
alto,0.071415
bajo,0.083916
medio,0.085400


**Conclusión 3ra  hipótesis**
Las personas con **ingresos medios tienen una tasa de impago más alta!!** 
y como se esperaba las personas con mayores ingresos son las que mayor pagan a tiempo!!

**4ta hipótesis ¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [125]:
df.pivot_table(index='clean_purpose', values='debt', aggfunc= 'mean').sort_values(by='debt', ascending=True)

,debt
clean_purpose,
Property,0.072347
Wedding,0.080034
Education,0.092223
Car,0.093590


**Conclusion**
En promedio los mayores deudores son los que hacen un credito para un carro!!

# Conclusión general 

**Realizamos una exploración inicial de los datos:**

Viendo las siguientes inconsistencias;

- En la columna de `days_employed`, no tienen sentido los valores negativos y el entero máximo (son muchísimos los días para que una persona pueda trabajar en vida)
- En la columna `children` el valor -1 tampoco tiene sentido
- valores ausentes en `days_employed` y `total_income`
- valores simétricos en la columna 'days_employed' y 'total_income'.
- mala captura a la hora del registro de los datos 
- El porcentaje de valores ausentes fue del 10%


**Realizamos una transformacion de los datos:**
- corrregimos errores tipográficos y de ortografia
- cambiamos valores que no tenian ningun sentido probablemente por errores de captura
- Eliminamos datos duplicados
- Agregamos valores más descriptivos a ciertas columnas
- Añadimos nuevas caracteristicas a nuestra tabla
- Trabajamos con los valores ausentes


**Trabajamos con las hipótesis:**

Pudimos evidenciar que las mayores tasas de impago son: 

- Las personas con rango de 1 a 2 hijos.
- Las personas Solteras
- Las personas con ingresos medios
- Las personas con credito para un carro





